https://thelaziestprogrammer.com/sharrington/math-of-machine-learning/solving-logreg-newtons-method

https://rstudio-pubs-static.s3.amazonaws.com/160015_b192ca9855e84b57814e785ebd034a5e.html

https://www.dotnetlovers.com/Article/225/logistic-regression-explained

https://www.kaggle.com/code/elyas19/implement-logistic-regression-from-scratch/notebook

https://sonsnotation.blogspot.com/2020/11/2-logistic-regression.html

https://lee-jaejoon.github.io/stat-logistic/

In [54]:
import numpy as np
import pandas as pd
import random

# Data Generating

In [64]:
sample_size = 100
x = np.random.normal(0,1,sample_size)
pi = 0.3 + (1 * x)
p = 1 / (1 + np.exp(-pi))
y = np.random.binomial(1,p,sample_size) # 각각 매칭이 되는지
df = pd.DataFrame(list(zip(np.ones(100,int),x, y)),columns=['intercept','x','y'])

In [65]:
df

,intercept,x,y
0,1,0.362937,1
1,1,-1.606296,0
2,1,0.358082,1
3,1,0.188906,0
4,1,-1.189291,0
...,...,...,...
95,1,-1.323655,0
96,1,0.856513,0
97,1,-0.225662,1
98,1,1.917120,0


# Newton's Method

In [66]:
# x = np.concatenate((df['x'].values.reshape(-1,1),df['intercept'].values.reshape(-1,1)), axis=1)
x = df[['intercept','x']].values
y = df['y'].values.reshape(-1,1)
weight_vec = np.random.normal(0,1,[2,1])
weight = [weight_vec]
while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = x.T@(pi-y)
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = x.T.dot(np.diag(pi.reshape(len(df)))).dot(np.diag((1-pi).reshape(len(df)))).dot(x)
    weight_vec = weight_vec - (1/len(df))*(np.linalg.inv(H).T@grad) # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))
    
    if all(np.round(weight[-2],5) == np.round(weight_vec,5)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        XWX = np.linalg.inv(H)
        break
        

[[1.94]
 [2.05]]
[[1.9 ]
 [2.03]]
[[1.86]
 [2.  ]]
[[1.83]
 [1.98]]
[[1.8 ]
 [1.96]]
[[1.76]
 [1.94]]
[[1.73]
 [1.92]]
[[1.7]
 [1.9]]
[[1.67]
 [1.88]]
[[1.64]
 [1.86]]
[[1.62]
 [1.84]]
[[1.59]
 [1.83]]
[[1.57]
 [1.81]]
[[1.54]
 [1.8 ]]
[[1.52]
 [1.78]]
[[1.49]
 [1.77]]
[[1.47]
 [1.76]]
[[1.45]
 [1.75]]
[[1.43]
 [1.73]]
[[1.41]
 [1.72]]
[[1.39]
 [1.71]]
[[1.37]
 [1.7 ]]
[[1.35]
 [1.69]]
[[1.33]
 [1.68]]
[[1.31]
 [1.67]]
[[1.29]
 [1.67]]
[[1.28]
 [1.66]]
[[1.26]
 [1.65]]
[[1.24]
 [1.64]]
[[1.22]
 [1.63]]
[[1.21]
 [1.63]]
[[1.19]
 [1.62]]
[[1.18]
 [1.61]]
[[1.16]
 [1.61]]
[[1.15]
 [1.6 ]]
[[1.13]
 [1.59]]
[[1.12]
 [1.59]]
[[1.1 ]
 [1.58]]
[[1.09]
 [1.58]]
[[1.08]
 [1.57]]
[[1.06]
 [1.57]]
[[1.05]
 [1.56]]
[[1.04]
 [1.56]]
[[1.03]
 [1.55]]
[[1.01]
 [1.55]]
[[1.  ]
 [1.54]]
[[0.99]
 [1.54]]
[[0.98]
 [1.53]]
[[0.97]
 [1.53]]
[[0.96]
 [1.53]]
[[0.94]
 [1.52]]
[[0.93]
 [1.52]]
[[0.92]
 [1.52]]
[[0.91]
 [1.51]]
[[0.9 ]
 [1.51]]
[[0.89]
 [1.51]]
[[0.88]
 [1.5 ]]
[[0.87]
 [1.5 ]]
[[0.86]
 [1.5 ]]

# Predict

In [15]:
df['p'] = 1/(1+np.exp(-(weight_vec[0]*df['x'] + weight_vec[1])))
df['p'].fillna(0,inplace=True)

# cutoff
df.loc[df['p'] >= 0.5, 'result'] = 1
df.loc[df['p'] < 0.5, 'result'] = 0

In [16]:
df

,intercept,x,y,p,result
0,1,-0.411190,1,0.538468,1.0
1,1,-0.393885,1,0.542018,1.0
2,1,0.065060,1,0.633555,1.0
3,1,-0.169354,1,0.587564,1.0
4,1,-1.933034,0,0.249239,0.0
...,...,...,...,...,...
95,1,1.696204,1,0.869282,1.0
96,1,-0.625506,1,0.494295,0.0
97,1,-0.735416,1,0.471632,0.0
98,1,-1.023797,0,0.412956,0.0


# Library

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [43]:
model = LogisticRegression()
model.fit(df['x'].values.reshape(-1,1), df["y"])

LogisticRegression()

# Comparison

In [44]:
# Coefficient
print("Implementation:",weight[-1].T)
print("sklearn library:",model.intercept_,model.coef_)

Implementation: [[0.36263258 1.05376515]]
sklearn library: [0.35423379] [[0.97681237]]


In [21]:
# Score
pred = model.predict(df['x'].values.reshape(-1,1))
print("Implementation:",sum(df.result.values == df.y.values)/1)
print("sklearn library:",np.round(accuracy_score(df['y'],pred)*100,2))

Implementation: 66.0
sklearn library: 66.0


# Variance and Standard Error of $ \hat{\beta} $

In [45]:
np.diag(np.sqrt(XWX))

array([0.22316317, 0.28283268])

In [47]:
import statsmodels.api as sm
logit_model = sm.Logit(y, df[['intercept','x']]).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.597306
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                          Logit   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Fri, 13 May 2022   Pseudo R-squ.:                  0.1259
Time:                        17:46:16   Log-Likelihood:                -59.731
converged:                       True   LL-Null:                       -68.331
Covariance Type:            nonrobust   LLR p-value:                 3.361e-05
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.3630      0.223      1.627      0.104      -0.074       0.800
x              1.0529      0.283      3.724      0.000       0.499       1.607
==============================================================================
"""

# Application of N-dimensional

In [24]:
sample_size = 100
x1 = np.random.normal(0,1,sample_size)
x2 = np.random.normal(0,1,sample_size)
pi = 0.3 + ((1 * x1)+(0.4 * x2))
p = 1 / (1 + np.exp(-pi))
y = np.random.binomial(1,p,sample_size)
df = pd.DataFrame(list(zip(np.ones(100,int),x1, x2, y)),columns=['intercept','x1','x2','y'])

In [26]:
# x = np.concatenate((df['x'].values.reshape(-1,1),df['intercept'].values.reshape(-1,1)), axis=1)
x = df[['intercept','x1','x2',]].values
y = df['y'].values.reshape(-1,1)
weight_vec = np.random.normal(0,1,[3,1])
weight = [weight_vec]

while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = (1/len(df))*x.T@(pi-y)
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = (1/len(df))*x.T.dot(np.diag(pi.reshape(len(df)))).dot(np.diag((1-pi).reshape(len(df)))).dot(x)
    weight_vec = weight_vec - np.linalg.inv(H).T@grad # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))
    
    if all(np.round(weight[-2],10) == np.round(weight_vec,10)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        break

[[0.54]
 [1.  ]
 [0.37]]
[[0.6 ]
 [1.08]
 [0.4 ]]
[[0.61]
 [1.09]
 [0.4 ]]
[[0.61]
 [1.09]
 [0.4 ]]
[[0.61]
 [1.09]
 [0.4 ]]
[[0.61]
 [1.09]
 [0.4 ]]


In [36]:
model = LogisticRegression()
model.fit(df[['x1','x2',]].values.reshape(-1,2), df["y"])
print("sklearn library:", model.intercept_, model.coef_)
print("Implementation:",weight[-1].T)

sklearn library: [0.58692131] [[1.01128142 0.38149259]]
Implementation: [[0.6070931  1.08560023 0.4004788 ]]


In [207]:
import pandas as pd
from sklearn import datasets

# 유방암 데이터셋 로드
data = datasets.load_breast_cancer()
ex = pd.DataFrame(data.data, columns = data.feature_names)
ex['target'] = data.target
ex['intercept'] = 1
ex = ex[['intercept','mean radius', 'mean texture', 'mean area', 'mean symmetry','target']]
ex

,intercept,mean radius,mean texture,mean area,mean symmetry,target
0,1,17.99,10.38,1001.0,0.2419,0
1,1,20.57,17.77,1326.0,0.1812,0
2,1,19.69,21.25,1203.0,0.2069,0
3,1,11.42,20.38,386.1,0.2597,0
4,1,20.29,14.34,1297.0,0.1809,0
...,...,...,...,...,...,...
564,1,21.56,22.39,1479.0,0.1726,0
565,1,20.13,28.25,1261.0,0.1752,0
566,1,16.60,28.08,858.1,0.1590,0
567,1,20.60,29.33,1265.0,0.2397,0


In [216]:
x = ex[['intercept','mean radius', 'mean texture', 'mean area', 'mean symmetry']].values
y = ex['target'].values.reshape(-1,1)
weight_vec =np.random.normal(size=(5,1))
#weight_vec = np.zeros((5,1))
#weight_vec = np.ones((5,1))
weight = [weight_vec]

while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = x.T@(pi-y)
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = x.T.dot(np.diag(pi.reshape(len(ex)))).dot(np.diag((1-pi).reshape(len(ex)))).dot(x)
    weight_vec = weight_vec - (1/len(ex))*(np.linalg.inv(H).T@grad) # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))

    if all(np.round(weight[-2],5) == np.round(weight_vec,5)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        break

[[ 0.89]
 [-0.53]
 [ 0.28]
 [-0.01]
 [-0.71]]
[[ 1.58]
 [-0.64]
 [ 0.27]
 [-0.01]
 [-0.99]]
[[ 2.09]
 [-0.71]
 [ 0.26]
 [-0.  ]
 [-1.25]]
[[ 2.48]
 [-0.77]
 [ 0.25]
 [-0.  ]
 [-1.5 ]]
[[ 2.78]
 [-0.8 ]
 [ 0.24]
 [-0.  ]
 [-1.74]]
[[ 3.  ]
 [-0.83]
 [ 0.24]
 [-0.  ]
 [-1.96]]
[[ 3.17]
 [-0.84]
 [ 0.23]
 [-0.  ]
 [-2.16]]
[[ 3.29]
 [-0.85]
 [ 0.22]
 [-0.  ]
 [-2.35]]
[[ 3.38]
 [-0.85]
 [ 0.22]
 [ 0.  ]
 [-2.54]]
[[ 3.44]
 [-0.85]
 [ 0.21]
 [ 0.  ]
 [-2.71]]
[[ 3.49]
 [-0.84]
 [ 0.21]
 [ 0.  ]
 [-2.87]]
[[ 3.51]
 [-0.83]
 [ 0.2 ]
 [ 0.  ]
 [-3.02]]
[[ 3.53]
 [-0.83]
 [ 0.2 ]
 [ 0.  ]
 [-3.16]]
[[ 3.53]
 [-0.81]
 [ 0.19]
 [ 0.  ]
 [-3.29]]
[[ 3.52]
 [-0.8 ]
 [ 0.19]
 [ 0.  ]
 [-3.42]]
[[ 3.51]
 [-0.79]
 [ 0.18]
 [ 0.  ]
 [-3.54]]
[[ 3.49]
 [-0.78]
 [ 0.18]
 [ 0.  ]
 [-3.66]]
[[ 3.47]
 [-0.76]
 [ 0.18]
 [ 0.  ]
 [-3.77]]
[[ 3.44]
 [-0.75]
 [ 0.17]
 [ 0.  ]
 [-3.88]]
[[ 3.41]
 [-0.73]
 [ 0.17]
 [ 0.  ]
 [-3.98]]
[[ 3.38]
 [-0.72]
 [ 0.17]
 [ 0.  ]
 [-4.07]]
[[ 3.35]
 [-0.7 ]
 [ 0.16]
 [ 0.  

[[-0.38]
 [ 0.52]
 [ 0.01]
 [-0.01]
 [-9.2 ]]
[[-0.39]
 [ 0.52]
 [ 0.01]
 [-0.01]
 [-9.22]]
[[-0.39]
 [ 0.52]
 [ 0.01]
 [-0.01]
 [-9.23]]
[[-0.4 ]
 [ 0.53]
 [ 0.01]
 [-0.01]
 [-9.25]]
[[-0.4 ]
 [ 0.53]
 [ 0.01]
 [-0.01]
 [-9.26]]
[[-0.41]
 [ 0.53]
 [ 0.01]
 [-0.01]
 [-9.28]]
[[-0.41]
 [ 0.53]
 [ 0.01]
 [-0.01]
 [-9.29]]
[[-0.42]
 [ 0.54]
 [ 0.01]
 [-0.01]
 [-9.3 ]]
[[-0.42]
 [ 0.54]
 [ 0.01]
 [-0.01]
 [-9.32]]
[[-0.43]
 [ 0.54]
 [ 0.01]
 [-0.01]
 [-9.33]]
[[-0.43]
 [ 0.54]
 [ 0.01]
 [-0.01]
 [-9.35]]
[[-0.44]
 [ 0.55]
 [ 0.01]
 [-0.01]
 [-9.36]]
[[-0.44]
 [ 0.55]
 [ 0.01]
 [-0.01]
 [-9.38]]
[[-0.45]
 [ 0.55]
 [ 0.01]
 [-0.01]
 [-9.39]]
[[-0.45]
 [ 0.55]
 [ 0.01]
 [-0.01]
 [-9.41]]
[[-0.45]
 [ 0.56]
 [ 0.01]
 [-0.01]
 [-9.42]]
[[-0.46]
 [ 0.56]
 [ 0.01]
 [-0.01]
 [-9.44]]
[[-0.46]
 [ 0.56]
 [ 0.01]
 [-0.01]
 [-9.45]]
[[-0.47]
 [ 0.56]
 [ 0.01]
 [-0.01]
 [-9.47]]
[[-0.47]
 [ 0.57]
 [ 0.01]
 [-0.01]
 [-9.48]]
[[-0.48]
 [ 0.57]
 [ 0.01]
 [-0.01]
 [-9.49]]
[[-0.48]
 [ 0.57]
 [ 0.01]
 [-0.01

[[ -0.64]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.76]]
[[ -0.64]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.78]]
[[ -0.64]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.79]]
[[ -0.64]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.81]]
[[ -0.63]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.83]]
[[ -0.63]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.84]]
[[ -0.63]
 [  0.91]
 [ -0.03]
 [ -0.02]
 [-12.86]]
[[ -0.63]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.87]]
[[ -0.63]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.89]]
[[ -0.62]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.91]]
[[ -0.62]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.92]]
[[ -0.62]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.94]]
[[ -0.62]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.95]]
[[ -0.62]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.97]]
[[ -0.61]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-12.99]]
[[ -0.61]
 [  0.92]
 [ -0.03]
 [ -0.02]
 [-13.  ]]
[[ -0.61]
 [  0.93]
 [ -0.03]
 [ -0.02]
 [-13.02]]
[[ -0.61]
 [  0.93]
 [ -0.03]
 [ -0.02]
 [-13.04]]
[[ -0.61]
 [  0.93]
 [ -0.03]
 [ -0.02]
 [-13.05]]
[[ -0.6 ]
 [  0.93]
 [ -0.03]
 

[[  0.18]
 [  1.1 ]
 [ -0.06]
 [ -0.02]
 [-16.79]]
[[  0.18]
 [  1.1 ]
 [ -0.06]
 [ -0.02]
 [-16.81]]
[[  0.19]
 [  1.1 ]
 [ -0.06]
 [ -0.02]
 [-16.82]]
[[  0.19]
 [  1.1 ]
 [ -0.06]
 [ -0.02]
 [-16.84]]
[[  0.2 ]
 [  1.1 ]
 [ -0.06]
 [ -0.02]
 [-16.86]]
[[  0.2 ]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.88]]
[[  0.21]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.9 ]]
[[  0.21]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.92]]
[[  0.22]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.94]]
[[  0.22]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.95]]
[[  0.23]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.97]]
[[  0.23]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-16.99]]
[[  0.24]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.01]]
[[  0.24]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.03]]
[[  0.25]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.05]]
[[  0.25]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.07]]
[[  0.26]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.08]]
[[  0.27]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.1 ]]
[[  0.27]
 [  1.11]
 [ -0.06]
 [ -0.02]
 [-17.12]]
[[  0.28]
 [  1.11]
 [ -0.06]
 

[[ 1.630e+00]
 [ 1.200e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.128e+01]]
[[ 1.64e+00]
 [ 1.20e+00]
 [-8.00e-02]
 [-2.00e-02]
 [-2.13e+01]]
[[ 1.650e+00]
 [ 1.200e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.132e+01]]
[[ 1.650e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.134e+01]]
[[ 1.660e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.136e+01]]
[[ 1.670e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.138e+01]]
[[ 1.68e+00]
 [ 1.21e+00]
 [-8.00e-02]
 [-2.00e-02]
 [-2.14e+01]]
[[ 1.680e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.142e+01]]
[[ 1.690e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.144e+01]]
[[ 1.700e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.146e+01]]
[[ 1.710e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.149e+01]]
[[ 1.710e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.151e+01]]
[[ 1.720e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.153e+01]]
[[ 1.730e+00]
 [ 1.210e+00]
 [-8.000e-02]
 [-2.000e-02]
 [-2.155e+01]]
[[ 1.740e+00]
 [

...

[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.558e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.150e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.100e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.1

[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.1

[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.1

[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.1

[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.151e+01]
 [ 5.000e-01]
 [-2.700e-01]
 [-2.000e-02]
 [-5.559e+01]]
[[ 2.1

In [217]:
model = LogisticRegression()
model.fit(ex[['mean radius', 'mean texture', 'mean area', 'mean symmetry']].values.reshape(-1,4), ex["target"])
print("sklearn library:", model.intercept_, model.coef_)

sklearn library: [0.70863688] [[ 1.60363641 -0.18989728 -0.03011479 -1.40789186]]


In [19]:
ex

,intercept,mean radius,mean texture,mean area,mean symmetry,target
0,1,17.99,10.38,1001.0,0.2419,0
1,1,20.57,17.77,1326.0,0.1812,0
2,1,19.69,21.25,1203.0,0.2069,0
3,1,11.42,20.38,386.1,0.2597,0
4,1,20.29,14.34,1297.0,0.1809,0
...,...,...,...,...,...,...
564,1,21.56,22.39,1479.0,0.1726,0
565,1,20.13,28.25,1261.0,0.1752,0
566,1,16.60,28.08,858.1,0.1590,0
567,1,20.60,29.33,1265.0,0.2397,0
